In [30]:
from fastapi import FastAPI

import numpy as np
import pandas as pd
import uvicorn

In [31]:
app = FastAPI()
genreFunction = pd.read_parquet('./data/genre_functions.parquet')

In [52]:
genreFunction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761461 entries, 0 to 9761460
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   genres            object
 1   year              int16 
 2   user_id           object
 3   playtime_forever  int32 
dtypes: int16(1), int32(1), object(2)
memory usage: 204.8+ MB


In [53]:
userRecommend = pd.read_parquet('./data/recommendations.parquet')
userRecommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5948940 entries, 0 to 5948939
Data columns (total 5 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   title               object
 1   playtime_forever    int32 
 2   year                int16 
 3   recommend           bool  
 4   sentiment_analysis  int8  
dtypes: bool(1), int16(1), int32(1), int8(1), object(1)
memory usage: 90.8+ MB


In [34]:
userRecommend.drop(['id', 'user_id'], axis = 1, inplace=True)
userRecommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5948940 entries, 0 to 5948939
Data columns (total 5 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   title               object
 1   playtime_forever    int32 
 2   year                int16 
 3   recommend           bool  
 4   sentiment_analysis  int8  
dtypes: bool(1), int16(1), int32(1), int8(1), object(1)
memory usage: 90.8+ MB


In [35]:
items = pd.read_parquet('./data/items.parquet')
items

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0
...,...,...,...,...
5094077,76561198329548331,346330,BrainBread 2,0
5094078,76561198329548331,373330,All Is Dust,0
5094079,76561198329548331,388490,One Way To Die: Steam Edition,3
5094080,76561198329548331,521570,You Have 10 Seconds 2,4


In [36]:

# We apply the str.lower() method so that it does not matter how the word is written.
genreFunction['genres'] = genreFunction['genres'].str.lower()
genreFunction['genres'].unique()

array(['action', 'indie', 'racing', 'strategy', 'rpg', 'simulation',
       'casual', 'adventure', 'sports', 'free to play',
       'massively multiplayer', 'early access',
       'animation &amp; modeling', 'education', 'software training',
       'utilities', 'web publishing', 'design &amp; illustration',
       'audio production', 'video production', 'photo editing'],
      dtype=object)

In [37]:
genreFunction

,genres,year,user_id,playtime_forever
0,action,1997,UTNerd24,5
1,action,1997,I_DID_911_JUST_SAYING,0
2,action,1997,76561197962104795,0
3,action,1997,r3ap3r78,0
4,action,1997,saint556,13
...,...,...,...,...
9761456,action,2004,76561198273508956,0
9761457,action,2004,76561198282090798,0
9761458,action,2004,943525,0
9761459,action,2004,76561198283312749,9


In [54]:
# mostPlayed = genreFunction.groupby('user_id')['playtime_forever'].sum()
# mostPlayed.head()

In [55]:
# The app.get decorator is used to associate the function with an HTTP GET request.
# '/PlayTimeGenre/{genre}/' is the path for the endpoint, the {genre} is the parameter.
@app.get('/PlayTimeGenre/{genre}/')
def PlayTimeGenre( genre : str ):
    '''
    Find the genre with the most playtime hours.

    Parameters
    ----------
    genre : str
        Desired genre to look for.
    
    Returns
    -------
    int
        Year when the highest number of hours played was recorded for that genre.
    
    Examples
    -------
    >>> PlayTimeGenre('Action')
    2012
    >>> PlayTimeGenre('Indie')
    2006

    '''
    # We look in our DataSet if there is any match with the genre entered.
    genres = genreFunction[genreFunction['genres'] == genre.lower()]
    if genres.empty:
        return f'The genre {genre} does not exist.'

    # We calculate the total playtime for each year in the dataset.
    results = genres.groupby('year')['playtime_forever'].sum()
    
    # Locate the index where the maximum value is. In this case the most played genre.
    idResut = results.idxmax()
    
    return f'Year with the most playtime hours for {genre}: {idResut}'

In [56]:
test = PlayTimeGenre('RPG')
print(test)

Year with the most playtime hours for RPG: 2011


# Segunda Función

In [57]:
@app.get('/PlayTimeGenre/{genre}/')
def UserForGenre( genre : str ):
    '''
    Find the user with the most hours played by genre, as well as hours played for each year.

    Parameters
    ----------
    genre : str
        Desired genre to look for.
    
    Returns
    -------
    list
        The user name and the amount of hours played by year for that particular genre.
    
    Examples
    -------
    >>> UserForGenre('Simulation')
    UserName is the user with the most playtime for the genre "Action" with 23721 hours played.

    Year            2003    2006    2009    2010    2011    2012    2013    2014    2015    2016    
    Hours Played      0      0      2037    4102    1968     223     323    342     1224     112   
    '''
    
    
    # Look in the DataSet if there is any match with the genre entered.
    genres = genreFunction[genreFunction['genres'] == genre.lower()]

    if genres.empty:
        return f'The genre {genre} does not exist.'

    # Group the DataSet by user ID, then sum the amount of hours played.
    mostPlayed = genres.groupby('user_id')['playtime_forever'].sum().reset_index()
    
    # Locate the index for the player.
    player = genres.loc[genres['playtime_forever'].idxmax()]['user_id']

    # Filter the DataSet with only the player id
    filteredDFWithPlayerID = (genres[genres['user_id'] == player])

    # Create a new DF with just the year and playtime_forever columns
    hoursPlayedByYear = filteredDFWithPlayerID.groupby('year')['playtime_forever'].sum()
    
    
    print(f'{player} is the user with the most playtime for the genre "{genre.capitalize()}" with {filteredDFWithPlayerID["playtime_forever"].sum()} hours played.')
    print(f'\nYear\tHours Played\n')
    for year, hour in hoursPlayedByYear.items():
        print(f'{year}\t{hour}\n')

In [58]:
UserForGenre('actiON')

Evilutional is the user with the most playtime for the genre "Action" with 795347 hours played.

Year	Hours Played

2003	0

2006	0

2009	2037

2010	4102

2011	1968

2012	680991

2013	69726

2014	24226

2015	112

2016	1291

2017	10894



# 3rd Function

In [59]:
userRecommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5948940 entries, 0 to 5948939
Data columns (total 5 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   title               object
 1   playtime_forever    int32 
 2   year                int16 
 3   recommend           bool  
 4   sentiment_analysis  int8  
dtypes: bool(1), int16(1), int32(1), int8(1), object(1)
memory usage: 90.8+ MB


In [60]:
@app.get('/UsersRecommend/{year}/')
def UsersRecommend( year : int ):
    '''
    Get the three most recommended games  
    
    Parameters
    ----------
    year : str
        Year in which the top three recommended games are.
    
    Returns
    -------
    list
        Name of the three recommended games.
    
    Examples
    -------
    >>> UsersRecommend(2018):
    Counter-Strike: Global Offense, Garry's Mode, Fall Guys
    >>> UsersRecommend(2021)
    Empire: Total War, Left 4 Dead 2, The Stanley Parable
    '''

    # Create a DataSet with rows that match the year.
    givenYear = userRecommend[userRecommend['year'] == year]

    if givenYear.empty:
        return f'There are no records for the year {year}.'

    # Group the top three games
    topThree = (givenYear['title'].value_counts().head(3).reset_index()
        .rename(columns={ 'title': 'Game', 'count': 'Positive Reviews'}))
    

    return [{f'Top {i+1}: "{game}" with {reviews} positive reviews'} for i, (game, reviews) in topThree.iterrows()]


In [61]:
UsersRecommend(2000)

[{'Top 1: "Counter-Strike" with 10851 positive reviews'},
 {'Top 2: "Hitman: Codename 47" with 4966 positive reviews'},
 {'Top 3: "Deus Ex: Game of the Year Edition" with 4937 positive reviews'}]

# 4th Function

In [62]:
@app.get('/UsersNotRecommend/{year}/')
def UsersNotRecommend( year : int ):
    '''
    Get the three least recommended games.
    
    Parameters
    ----------
    year : str
        Year in which the three least recommended games are.
    
    Returns
    -------
    list
        Name of the three recommended games.
    
    Examples
    -------
    >>> UsersNotRecommend(2008):
    Portal 2, Garry's Mode, Fall Guys
    >>> UsersNotRecommend(2011)
    Carmageddon Max Pack, Left 4 Dead 2, The Stanley Parable
    '''

    givenYear = userRecommend[userRecommend['year'] == year]

    if givenYear.empty:
        return f'There are no records for the year {year}.'

    # Group the games that are from the desired year and had negative reviews.
    leastRecommendedGames = userRecommend[(userRecommend['year'] == year) & (userRecommend['recommend'] == False)]

    # Create a list that has the 3 least recommended games for that year.
    leastThree = (
        leastRecommendedGames['title']
        .value_counts()
        .head(3)
        .reset_index()
        .rename(columns={'count': 'Negative Reviews', 'title': 'Game'})
    )

    leastThree = [{f'Top {i+1}: "{game}" with {reviews} negative reviews'} for i, (game, reviews) in leastThree.iterrows()]

    return leastThree

In [63]:
UsersNotRecommend(2018)

[{'Top 1: "Lost Moon" with 40 negative reviews'},
 {'Top 2: "Starwalker" with 12 negative reviews'},
 {'Top 3: "Empires Of Creation" with 9 negative reviews'}]

In [64]:
userRecommend.head(1)

,title,playtime_forever,year,recommend,sentiment_analysis
0,Carmageddon Max Pack,5,1997,True,0


# 5th Function

In [65]:
@app.get('/sentiment_analysis/{year}/')
def sentiment_analysis( year : int ):
    '''
    Get the category reviews from all users in a year.
    
    Parameters
    ----------
    year : str
        Desired year to see how the reviews were.
    
    Returns
    -------
    list
        Amount of all the different review categories for that year.
    
    Examples
    -------
    >>> sentiment_analysis(2018)
    {Negative = 101, Neutral = 142, Positive = 221}
    >>>sentiment_analysis(2019)
    {Negative = 140, Neutral = 47, Positive = 115}
    '''

    givenYear = userRecommend[userRecommend['year'] == year]

    if givenYear.empty:
        return f'There are no records for the year {year}.'
    
    
    sentiment = userRecommend.groupby('year')['sentiment_analysis'].value_counts().unstack(fill_value=0)


    sentiment = sentiment.loc[year].to_dict()
     
    return {"Negative": sentiment.get(0, 0),
            "Neutral": sentiment.get(1, 0),
            "Positive": sentiment.get(2, 0)}
            


In [66]:
sentiment_analysis(2009)

{'Negative': 70585, 'Neutral': 59924, 'Positive': 175569}

In [67]:
sentiment_analysis(2018)

{'Negative': 101, 'Neutral': 142, 'Positive': 221}